In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bfb4ace981627e35b5f6e3d8b5cf4f2bd669d8c9ecdeff2df41d9cdf5d8ba33b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import requests

In [3]:
def download_file(url, filename):

    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully and saved as {filename}")
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"OOps: Something Else: {err}")

In [13]:
url = 'https://gist.githubusercontent.com/kevin336/acbb2271e66c10a5b73aacf82ca82784/raw/e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv'
filename = 'data.csv'

download_file(url, filename)

File downloaded successfully and saved as data.csv


In [14]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder.appName('DataFrame').getOrCreate()
spark

In [16]:
df_pyspark = spark.read.csv('data.csv', header=True, inferSchema=True)
df_pyspark.show(5)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

## GROUP BY and AGGREGRATE
work together

In [19]:
# max salary for for each job
df_pyspark.groupBy('JOB_ID').max('SALARY').show()

+----------+-----------+
|    JOB_ID|max(SALARY)|
+----------+-----------+
|FI_ACCOUNT|       9000|
|    MK_MAN|      13000|
|   IT_PROG|       9000|
|    FI_MGR|      12008|
|AC_ACCOUNT|       8300|
|    HR_REP|       6500|
|  PU_CLERK|       3100|
|    AC_MGR|      12008|
|    PR_REP|      10000|
|    ST_MAN|       8200|
|    MK_REP|       6000|
|    PU_MAN|      11000|
|  SH_CLERK|       2600|
|   AD_PRES|      24000|
|   AD_ASST|       4400|
|  ST_CLERK|       3600|
|     AD_VP|      17000|
+----------+-----------+



In [20]:
# how much salary each department gives
df_pyspark.groupBy('DEPARTMENT_ID').sum('SALARY').show()

+-------------+-----------+
|DEPARTMENT_ID|sum(SALARY)|
+-------------+-----------+
|           20|      19000|
|           40|       6500|
|          100|      51608|
|           10|       4400|
|           50|      85600|
|           70|      10000|
|           90|      58000|
|           60|      28800|
|          110|      20308|
|           30|      24900|
+-------------+-----------+



In [21]:
# count people for each position
df_pyspark.groupBy('JOB_ID').count().show()

+----------+-----+
|    JOB_ID|count|
+----------+-----+
|FI_ACCOUNT|    5|
|    MK_MAN|    1|
|   IT_PROG|    5|
|    FI_MGR|    1|
|AC_ACCOUNT|    1|
|    HR_REP|    1|
|  PU_CLERK|    5|
|    AC_MGR|    1|
|    PR_REP|    1|
|    ST_MAN|    5|
|    MK_REP|    1|
|    PU_MAN|    1|
|  SH_CLERK|    2|
|   AD_PRES|    1|
|   AD_ASST|    1|
|  ST_CLERK|   16|
|     AD_VP|    2|
+----------+-----+

